### Dataset Açıklaması
Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image.
n the 3-dimensional space is that described in: [K. P. Bennett and O. L. Mangasarian: "Robust Linear Programming Discrimination of Two Linearly Inseparable Sets", Optimization Methods and Software 1, 1992, 23-34].

This database is also available through the UW CS ftp server:
ftp ftp.cs.wisc.edu
cd math-prog/cpo-dataset/machine-learn/WDBC/

Also can be found on UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

Attribute Information:

1. ID number
2. Diagnosis (M = malignant, B = benign)
3-32)

Ten real-valued features are computed for each cell nucleus:

* a) radius (mean of distances from center to points on the perimeter)
* b) texture (standard deviation of gray-scale values)
* c) perimeter
* d) area
* e) smoothness (local variation in radius lengths)
* f) compactness (perimeter^2 / area - 1.0)
* g) concavity (severity of concave portions of the contour)
* h) concave points (number of concave portions of the contour)
* i) symmetry
* j) fractal dimension ("coastline approximation" - 1)

The mean, standard error and "worst" or largest (mean of the three
largest values) of these features were computed for each image,
resulting in 30 features. For instance, field 3 is Mean Radius, field
13 is Radius SE, field 23 is Worst Radius.

All feature values are recoded with four significant digits.

Missing attribute values: none

Class distribution: 357 benign, 212 malignant

### Kütüphaneleri Ekleyelim

In [ ]:
import numpy as np
import pandas as pd 
import os

%matplotlib inline 
import matplotlib.pyplot as plt

import seaborn as sns

from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

from sklearn import preprocessing
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, train_test_split, cross_val_predict, cross_val_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

from keras import Sequential
from keras.layers import Dense
from keras.optimizers import SGD


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

Verimizi yükleyelim.

In [ ]:
df = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')
df

### Dataset İncelemesi

İlk 5 gözlemi inceleyelim.

In [ ]:
df.head()

Rastgele 5 gözlemi inceleyelim.

In [ ]:
df.sample(5)

Veri setinin değişken ve gözlem sayısını öğrenelim.

In [ ]:
df.shape

### Veri Ön İşleme (Data Preprocessing)

In [ ]:
df.info()

Bilgileri incelersek ; 
* 33 kolonun olduğunu, kolonların tiplerini ve bellek kullanımlarını,
* 'Unnamed: 32' özniteliğinin tüm değerlerinin eksik olduğunu,
* Geri kalan tüm özniteliklerin eksiksiz olduğunuz öğrenebiliriz.

Verimizi düzenleyelim.

In [ ]:
df.drop('id',axis=1,inplace=True)
df.drop('Unnamed: 32',axis=1,inplace=True)

In [ ]:
df['diagnosis'] = df['diagnosis'].map({'M':1,'B':0})
df.head()

Veri setinin istatistik bilgilerini inceleyelim.

In [ ]:
df.describe().T

In [ ]:
plt.figure(figsize=(8,5))
sns.set_theme(style="darkgrid")
ax = sns.countplot(x="diagnosis", data=df)
plt.title("Diagnosis M=1 , B=0")
plt.show()

### Veriyi Ölçeklendirme (Scaling) ve Normalize Etme (Normalization)

Veriyi ölçeklendirmek için min-maks ölçeklendirme kullanalım.

In [ ]:
df.head(1)

In [ ]:
df_normalized=df.copy()

In [ ]:
for i in df.columns:
    x = df_normalized[[i]].values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_normalized[i] = pd.DataFrame(x_scaled)

In [ ]:
df_normalized.head(1)

In [ ]:
df.head(1)

### Veriyi İnceleyelim

In [ ]:
plt.figure(figsize=(20, 15))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(df_normalized.corr(), dtype=np.bool))
heatmap = sns.heatmap(df_normalized.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':25}, pad=16);

In [ ]:
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(df_normalized.corr()[['diagnosis']].sort_values(by='diagnosis', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Features Correlating with Diagnosis', fontdict={'fontsize':18}, pad=16);

In [ ]:
plt.scatter(df_normalized['concave points_worst'], df_normalized['perimeter_worst'], c=df_normalized['diagnosis'], cmap=plt.cm.Spectral)
plt.colorbar()
plt.xlabel('concave points_worst')
plt.ylabel('perimeter_worst')
plt.title('concave points_worst x perimeter_worst')
plt.show()

In [ ]:
features_mean=list(df.columns[1:])
# split dataframe into two based on diagnosis
dfM=df[df['diagnosis'] ==1]
dfB=df[df['diagnosis'] ==0]

In [ ]:
hist_data = [dfM['concave points_worst'],dfB['concave points_worst']]

group_labels = ['M', 'B']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.2)
fig.show()

In [ ]:
df.shape

In [ ]:
plt.rcParams.update({'font.size': 8})
fig, axes = plt.subplots(nrows=10, ncols=3, figsize=(15,20))
axes = axes.ravel()
for idx,ax in enumerate(axes):
    ax.figure
    binwidth= (max(df[features_mean[idx]]) - min(df[features_mean[idx]]))/50
    ax.hist([dfM[features_mean[idx]],dfB[features_mean[idx]]], bins=np.arange(min(df[features_mean[idx]]), max(df[features_mean[idx]]) + binwidth, binwidth) , alpha=0.5,stacked=True, label=['M','B'],color=['r','g'])
    ax.legend(loc='upper right')
    ax.set_title(features_mean[idx])
plt.tight_layout()
plt.show()

#### Çıkarımlar;
* Heatmap'de korelasyonlarını incelediğimizde diagnosis üzerinde en çok etkili olanların hangi değişkenler olduğunu tespit edebiliriz. 
* Histogramları incelerken M ve B değerlerinin yoğunlaşmalarının birbirlerine göre farklarını dikkate almalıyız. Eğer farklı yerlerde yoğunlaşıyorsa özniteliği kanser espitinde kullanabilriz. Örneği radius_mean özniteliğinin histogramındaki yoğunlaşma farklıyken, smoothness_mean özniteliğinin yoğunlaşması düzgündür.

### Cross Validation (KFold) Uygulaması

Bu yöntemde veri kümesi k sayısı kadar eşit parçaya bölünür ve her parçanın hem eğitim hem de test için kullanılması sağlanır. Bu sayede test kümesi seçimi yapılırken oluşan yanlılık ortadan kaldırılmış olur.

Öncelikle klasik yöntem ile veri setini train ve test olarak ayırarak bir doğrusal regresyon analizi yapalım.

In [ ]:
x = df_normalized.drop("diagnosis", axis=1)
y = df_normalized["diagnosis"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle = False, random_state = 0)

Ayırdığımız train test verilerini kullanarak modelimizi eğitelim

### Lojistik Regresyon (Logistic Regression)

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)

Modelimizi eğittik. Accuracy değerine bakalım.

In [ ]:
print("Accuracy Score: ",accuracy_score(y_test, y_pred))

Modeli farklı gözlemlerle kombinleyerek tekrardan skorunu hesaplayalım ve ortalamalarını alalım.

In [ ]:
print("Cross Validation Score: ",cross_val_score(lr, x_test, y_test, cv = 10).mean())

Klasik yöntem ile bölerek oluşturulan regresyon modelinin sonuçlarını inceledik. Şimdide çapraz doğrulama (Cross validation) yöntemiyle bölerek oluşturduğumuz verimizi modelimizde inceleyelim.

In [ ]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)
model = LogisticRegression()
scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
cross_val_score(lr, x_test, y_test, cv = 10).mean()
print('Accuracy: %.3f (%.3f)' % (scores.mean(), scores.std()))

In [ ]:
scores

### Karar Ağacı (Decision Tree)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtGini = DecisionTreeClassifier(random_state = 42, criterion='gini')
dtEntropy = DecisionTreeClassifier(random_state = 42, criterion='entropy')

cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(dtEntropy, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
cross_val_score(lr, x_test, y_test, cv = 10).mean()
print('Entropy Accuracy: %.3f (%.3f)' % (scores.mean(), scores.std()))

scores = cross_val_score(dtGini, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
cross_val_score(lr, x_test, y_test, cv = 10).mean()
print('Gini Accuracy: %.3f (%.3f)' % (scores.mean(), scores.std()))

In [ ]:
scores

### Yapay Sinir Ağları ( Artificial Neural Networks)

In [ ]:
NumerOfClasses = len(y_train.unique())
RN = Sequential() # create network structure
RN.add(Dense(10, input_shape = x_train.shape[1:], activation ='relu'))
RN.add(Dense(NumerOfClasses, activation ='sigmoid'))
RN.summary()

In [ ]:
from keras.utils import to_categorical
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9)
RN.compile(optimizer=sgd, loss='mean_squared_error', metrics=['accuracy'])
trainedRN = RN.fit(x_train, to_categorical(y_train), epochs=100, verbose=1)

In [ ]:
score = RN.evaluate(x_test, to_categorical(y_test),verbose=0)
print('Test Score:', score[0])
print('Test Accuracy:', score[1])